# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../output_data/Weath_Data.csv"
wd_df = pd.read_csv(csv_path)


wd_df = wd_df.drop(wd_df.columns[[0]], axis=1)
wd_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,qaanaaq,77.48,-69.36,39.09,73,100,8.50,GL,1592665442
1,mataura,-46.19,168.86,39.00,83,54,3.00,NZ,1592665442
2,komsomolskiy,40.43,71.72,86.00,25,0,2.24,UZ,1592665442
3,busselton,-33.65,115.33,44.01,67,0,7.20,AU,1592665312
4,rikitea,-23.12,-134.97,75.88,82,77,14.85,PF,1592665443
...,...,...,...,...,...,...,...,...,...
559,hidalgo,20.50,-99.00,65.57,69,36,2.75,MX,1592665534
560,alto longa,-5.25,-42.21,88.61,51,26,8.46,BR,1592665534
561,havelock,34.88,-76.90,81.00,78,5,3.36,US,1592665474
562,sioux lookout,50.10,-91.92,60.80,67,75,5.82,CA,1592665534


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = wd_df[["Lat", "Lng"]]
humidity = wd_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig
#plt.savefig('../output_data/heatmap.png')

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#A max temperature lower than 80 degrees but higher than 60.
clean_df = wd_df.loc[wd_df['Max Temp'] < 80]
clean_df.head()
clean_df = clean_df.loc[clean_df['Max Temp'] > 60]
clean_df.head()
# #Fahrenheit = (Celsius * 9/5) + 32

#Wind speed less than 20 mph.
clean_df = clean_df.loc[clean_df['Wind Speed'] > 2]
clean_df.head()

# #20% or less cloudiness.
clean_df = clean_df.loc[clean_df['Cloudiness'] >= 20]
hotel_df = clean_df.reset_index(drop=True)
#hotel_df = clean_df.dropna(inplace = True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,75.88,82,77,14.85,PF,1592665443
1,cape town,-33.93,18.42,64.00,88,75,12.75,ZA,1592665237
2,hasaki,35.73,140.83,70.00,88,75,5.82,JP,1592665307
3,saint-philippe,-21.36,55.77,71.60,68,81,5.82,RE,1592665446
4,kapaa,22.08,-159.32,77.00,83,75,16.11,US,1592665447
...,...,...,...,...,...,...,...,...,...
157,high level,58.52,-117.14,64.40,52,40,9.17,CA,1592665532
158,shanhetun,44.70,127.20,62.64,91,97,8.28,CN,1592665533
159,arona,28.10,-16.68,78.80,54,20,28.86,ES,1592665533
160,hidalgo,20.50,-99.00,65.57,69,36,2.75,MX,1592665534


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel = ""
hotel_df['Hotel Name'] = hotel


In [ ]:
#params = {"key": g_key,
         # 'type': 'hotel',
         # 'keyword': 'hotel',
         # 'radius': 5000}

In [7]:
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_radius = 5000
target_type = "lodging"
params = {"location": target_coordinates,
          "type": target_type,
          "radius": target_radius,
          "key": g_key}


In [21]:
# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']

    # update address key value
    params['location'] = f"{lat},{lng}"

    # make request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    #print(json.dumps(results, indent=4, sort_keys=True))
    
    try: 
        print(f"The hotel closest to {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
            print('Missing results...Skipping.')


Retrieving Results for Index 0: rikitea.
The hotel closest to rikitea is Pension Maro'i.
Retrieving Results for Index 1: cape town.
The hotel closest to cape town is Southern Sun Waterfront Cape Town.
Retrieving Results for Index 2: hasaki.
The hotel closest to hasaki is Inubosaki Kanko Hotel.
Retrieving Results for Index 3: saint-philippe.
The hotel closest to saint-philippe is Chambres d'hôte "La Trinité".
Retrieving Results for Index 4: kapaa.
The hotel closest to kapaa is Sheraton Kauai Resort at Coconut Beach.
Retrieving Results for Index 5: atuona.
The hotel closest to atuona is Villa Enata.
Retrieving Results for Index 6: livingstonia.
The hotel closest to livingstonia is Stone House Guest House.
Retrieving Results for Index 7: san quintin.
The hotel closest to san quintin is Old Mill Hotel.
Retrieving Results for Index 8: cidreira.
The hotel closest to cidreira is Novomar Pousada e Turismo Ltda.
Retrieving Results for Index 9: shingu.
The hotel closest to shingu is Shingu UI Ho

The hotel closest to winnemucca is Winners Inn Casino.
Retrieving Results for Index 87: lypnyazhka.
Missing results...Skipping.
Retrieving Results for Index 88: muros.
The hotel closest to muros is Apartment Av. Castelao.
Retrieving Results for Index 89: katsuura.
The hotel closest to katsuura is Katsuura Hotel Mikazuki.
Retrieving Results for Index 90: husavik.
The hotel closest to husavik is Bjarnabúð.
Retrieving Results for Index 91: naze.
The hotel closest to naze is 奄美ポートタワーホテル.
Retrieving Results for Index 92: ornskoldsvik.
The hotel closest to ornskoldsvik is Scandic Örnsköldsvik.
Retrieving Results for Index 93: vicuna.
The hotel closest to vicuna is Los Mosaicos.
Retrieving Results for Index 94: auki.
The hotel closest to auki is Auki Motel.
Retrieving Results for Index 95: lufilufi.
The hotel closest to lufilufi is Leuaina Beach Resort and Spa.
Retrieving Results for Index 96: aksum.
The hotel closest to aksum is Remhay Hotel.
Retrieving Results for Index 97: ribeira brava.
T

In [22]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,rikitea,-23.12,-134.97,75.88,82,77,14.85,PF,1592665443,Pension Maro'i
1,cape town,-33.93,18.42,64.00,88,75,12.75,ZA,1592665237,Southern Sun Waterfront Cape Town
2,hasaki,35.73,140.83,70.00,88,75,5.82,JP,1592665307,Inubosaki Kanko Hotel
3,saint-philippe,-21.36,55.77,71.60,68,81,5.82,RE,1592665446,"Chambres d'hôte ""La Trinité"""
4,kapaa,22.08,-159.32,77.00,83,75,16.11,US,1592665447,Sheraton Kauai Resort at Coconut Beach


In [32]:
# NOTE: Do not change any of the code in this cell
#locations = [hotel['Hotel Name'] for hotel in hotel_df] 
#locations
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
locations = [hotel[hotel_df[['Lat']['Lng']]] for hotel in hotel_df]
hotel_info = [info_box_template.format(**locations) for index, row in hotel_df.iterrows()]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

TypeError: list indices must be integers or slices, not str

In [ ]:
# Add marker layer ontop of heat map


# Display figure
